In [4]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
import threading
lock = threading.RLock()

In [2]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,41.79.197.150,8080,SO,Somalia,elite proxy,no,yes,1 minute ago
66,93.78.205.197,57437,UA,Ukraine,elite proxy,no,yes,4 minutes ago
63,113.130.126.212,57312,KH,Cambodia,elite proxy,no,yes,4 minutes ago
62,118.173.232.219,57499,TH,Thailand,elite proxy,no,yes,4 minutes ago
61,176.196.240.34,53061,RU,Russian Federation,elite proxy,no,yes,4 minutes ago


In [3]:
# eventsDf = pd.DataFrame(columns=[
#     'name',
#     'eventType',
#     'date',
#     'city',
#     'country',
#     'hasWebcast',
#     'hasSpace',
#     'eventCode',
# ])
eventsDf = pd.read_pickle('../datasets/rawEvents.pkl')
print(len(eventsDf), 'events')
eventsDf.head()

10106 events


,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode
32790,TEDxRiodelaPlata,Standard,"May 7, 2019",Ciudad Autónoma de Buenos Aires,Argentina,True,False,32790
33511,TEDxEsquel,Standard,"May 4, 2019",Esquel,Argentina,True,False,33511
31229,TEDxRafaela,Standard,"May 4, 2019",Rafaela,Argentina,True,False,31229
34453,TEDxBarilocheLive,TEDxLive,"April 27, 2019",San Carlos de Bariloche,Argentina,True,False,34453
34383,TEDxRosarioLive,TEDxLive,"April 18, 2019",Rosario,Argentina,True,False,34383


In [4]:
errors = pd.DataFrame(columns=['page','min','max','country'])

In [36]:
proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].sample(1).values

array(['91.135.148.198'], dtype=object)

In [1]:
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].values[0]
}
def getProxy():
    return {
        "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].sample(1).values
    }
def getUrl(url):
    return requests.get(
        url, headers=headersChrome, proxies=getProxy()
    )

def getEvents(searchFilter, startPage=1, maxPages=1, past=False):
    whenStr = 'upcoming'
    if(past):
        whenStr = 'past'
    if(type(maxPages) is not int):
        print('maxPages has to be int')
        return False
    for pageNum in range(startPage, maxPages+1):
#         searchFilter = urllib.parse.quote(searchFilter)
        url = 'https://www.ted.com/tedx/events?autocomplete_filter='+searchFilter+'&page='+str(pageNum)+'&when='+whenStr
        display(url)
        rawHtmlPage = getUrl(url)
        html = BeautifulSoup(rawHtmlPage.text, 'html.parser')
        eventsRows = html.select('tr.tedx-events-table__event')
        if(len(eventsRows) == 0):
            break
        for x in eventsRows:
            locationRaw = x.select('td')[2]
            ciudad, pais = re.findall('(.+)<br\/>(.+)<\/td>', str(locationRaw))[0]
            dateRaw = x.select('td:nth-of-type(1)')[0].text
            dateStr = re.findall('(\w+ [\d]+, [\d]+)', dateRaw)[0]
            a = x.select('strong a')[0]
            href = a['href']
            name = re.sub('\xad','', str(a.text))
            webcastRaw = x.select('td')[4]
            hasWebcast = re.findall('(.+)', str(webcastRaw))[3]!='No'

            spaceRaw = x.select('td')[3]
            hasSpace = re.findall('<span class="screen-reader-text">(.+)</span>', str(spaceRaw))[0] == 'Yes'
            eventCode = int(re.sub('/tedx/events/','', href))
            eventType = x.find_all(attrs={'class':'tedx-events-table__event-type'})[0].text
            with lock:
                eventsDf.loc[eventCode] = [
                    name,
                    eventType,
                    dateStr,
                    ciudad,
                    pais,
                    hasWebcast,
                    hasSpace,
                    eventCode,
                ]
    return True

NameError: name 'proxiesDf' is not defined

In [39]:
len(eventsDf)

11569

In [53]:
with open('../countries.json', 'r') as f:
    countries_dict = json.load(f)
    
countriesA = [countries_dict[x] for x in countries_dict]

countries = []
for c in countriesA:
    for y in range(2009, 2021):
        countries.append(c+'&year='+str(y))
len(countries)

2892

In [54]:
isPast = True
def process_id(country):
    """process a single ID"""
    global isPast
    country = re.sub(' ', '+', country)
    try:
        getEvents(country, 1, 1000, isPast)
    except:
        print('ERROR IN', country)
    return True

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = process_id(id)
    return store

from threading import Thread

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    return store

# countries = eventsDf.country.unique()
threaded_process_range(
    15,
    countries,
)

'https://www.ted.com/tedx/events?autocomplete_filter=Afghanistan&year=2010&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Afghanistan&year=2009&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Afghanistan&year=2011&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Afghanistan&year=2012&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Afghanistan&year=2013&page=1&when=past'

...

'https://www.ted.com/tedx/events?autocomplete_filter=Western+Sahara&year=2016&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Zimbabwe&year=2012&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Yemen&year=2019&page=1&when=past'

'https://www.ted.com/tedx/events?autocomplete_filter=Zimbabwe&year=2010&page=1&when=past'

{'Afghanistan&year=2009': True,
 'Afghanistan&year=2010': True,
 'Albania&year=2009': True,
 'Albania&year=2010': True,
 'Afghanistan&year=2011': True,
 'Afghanistan&year=2017': True,
 'Albania&year=2011': True,
 'Afghanistan&year=2020': True,
 'Afghanistan&year=2012': True,
 'Afghanistan&year=2014': True,
 'Afghanistan&year=2018': True,
 'Albania&year=2012': True,
 'Afghanistan&year=2015': True,
 'Afghanistan&year=2013': True,
 'Afghanistan&year=2016': True,
 'Afghanistan&year=2019': True,
 'Albania&year=2013': True,
 'Algeria&year=2012': True,
 'Albania&year=2014': True,
 'Albania&year=2020': True,
 'Algeria&year=2009': True,
 'Algeria&year=2013': True,
 'American Samoa&year=2011': True,
 'American Samoa&year=2015': True,
 'American Samoa&year=2012': True,
 'Algeria&year=2011': True,
 'American Samoa&year=2016': True,
 'Algeria&year=2014': True,
 'Algeria&year=2010': True,
 'Albania&year=2015': True,
 'Andorra&year=2018': True,
 'American Samoa&year=2014': True,
 'Albania&year=2019':

In [55]:
print(len(eventsDf), 'events')

12998 events


In [57]:
eventsDf[eventsDf.country=='Mexico']

,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode
31309,TEDxSecretariaDeEducacionDelEstadoDeZacatecas,Internal,"May 17, 2019",Guadalupe,Mexico,False,False,31309
31974,TEDxYouth@ASFM,Youth,"May 18, 2019",Santa Catarina,Mexico,False,True,31974
34319,TEDxTangamangaSalon,Salon,"May 24, 2019",San Luis Potosí,Mexico,False,True,34319
31493,TEDxEscuelaAmeyalli,Youth,"May 25, 2019",Mexico City,Mexico,False,False,31493
31829,TEDxINACIPE,University,"May 15, 2019",Ciudad de México,Mexico,True,False,31829
33087,TEDxUniversidadPanamericanaGuadalajara,University,"May 14, 2019",Guadalajara,Mexico,True,False,33087
33209,TEDxCiudadVictoria,Standard,"May 11, 2019",Ciudad Victoria,Mexico,True,False,33209
32986,TEDxHumaya,Standard,"May 8, 2019",Culiacán,Mexico,True,False,32986
33009,TEDxTecdeMtySanLuisPotosi,University,"May 3, 2019",San Luis Potosi,Mexico,True,False,33009
30425,TEDxBUAP,University,"May 2, 2019",Puebla,Mexico,True,False,30425


In [58]:
eventsDf.to_pickle('../datasets/rawEvents.pkl')